In [7]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report


In [2]:
# Load CSV files (adjust paths as needed)
data = pd.read_csv('UNSW_NB15_training-set.csv')
selected_features = [
    'dur', 'proto', 'service', 'state', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sload', 'dload',
    'spkts', 'dpkts', 'ct_srv_src', 'ct_srv_dst', 'smean', 'dmean', 'ct_dst_ltm', 'attack_cat',
]
data=data[selected_features]

In [3]:
# Identify categorical columns (object types) from selected features
categorical_columns = data.select_dtypes(include=['object']).columns.intersection(selected_features)

# Dictionary to keep track of encoders for each column
label_encoders = {}

# Apply Label Encoding where appropriate and store each encoder
for column in categorical_columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le  # Store the encoder

# Save the label encoders for future use
with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

# Separate features (X) and the target (y)
X = data.drop(columns=['attack_cat'])
y = data['attack_cat']

# Convert target to categorical if it’s multi-class
y_encoded = to_categorical(y)

# Normalize continuous features (optional but recommended for neural networks)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [5]:
# Define and compile the neural network model (same as before)
model = Sequential()
model.add(Dense(64, input_shape=(X_scaled.shape[1],), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(y_encoded.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10


C:\Users\Admin\anaconda3\envs\VulDet\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2059/2059 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.7127 - loss: 0.8818 - val_accuracy: 0.7855 - val_loss: 0.5850
Epoch 2/10
2059/2059 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7958 - loss: 0.5502 - val_accuracy: 0.8032 - val_loss: 0.5182
Epoch 3/10
2059/2059 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8036 - loss: 0.5165 - val_accuracy: 0.8083 - val_loss: 0.5147
Epoch 4/10
2059/2059 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8070 - loss: 0.5043 - val_accuracy: 0.7972 - val_loss: 0.5299
Epoch 5/10
2059/2059 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8118 - loss: 0.4920 - val_accuracy: 0.8142 - val_loss: 0.4843
Epoch 6/10
2059/2059 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8139 - loss: 0.4791 - val_accuracy: 0.8206 - val_loss: 0.4743
Epoch 7/10
2059/2059 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8157 - loss: 0.4750 - val_accuracy: 0.8147 - val_loss: 0.4763
Epoch 8/10
2059/2059 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8165 - loss: 0.4684 - val_accurac

In [8]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Predict the classes for the test data
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Classification report and confusion matrix
print("Classification Report:")
print(classification_report(y_test_classes, y_pred_classes))

print("Confusion Matrix:")
print(confusion_matrix(y_test_classes, y_pred_classes))

515/515 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8269 - loss: 0.4537
Test Loss: 0.4626
Test Accuracy: 0.8245
515/515 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       131
           1       0.00      0.00      0.00       117
           2       0.42      0.28      0.33       786
           3       0.65      0.75      0.70      2275
           4       0.51      0.29      0.37      1212
           5       0.97      0.98      0.98      3723
           6       0.88      0.96      0.92      7418
           7       0.65      0.77      0.70       723
           8       0.54      0.19      0.28        75
           9       0.00      0.00      0.00         7

    accuracy                           0.82     16467
   macro avg       0.46      0.42      0.43     16467
weighted avg       0.80      0.82      0.81     16467

Confusion Matrix:
[[   0    0   12   90   26    0    1    2   

C:\Users\Admin\anaconda3\envs\VulDet\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Admin\anaconda3\envs\VulDet\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Admin\anaconda3\envs\VulDet\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
model.save('IDS_Detect.h5')

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │             170 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,792 (46.07 KB)

 Trainable params: 3,930 (15.35 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 7,862 (30.71 KB)